In [ ]:
# Tutorial: https://colab.research.google.com/github/tensorflow/agents/blob/master/docs/tutorials/1_dqn_tutorial.ipynb#scrollTo=N7brXNIGWXjC

In [28]:
import numpy as np
import tensorflow as tf
import pyvirtualdisplay

from tf_agents.agents.dqn import dqn_agent
from tf_agents.environments import suite_gym
from tf_agents.specs import tensor_spec
from tf_agents.networks import sequential
from tf_agents.environments import tf_py_environment

In [29]:
env_name = 'CartPole-v0'
env = suite_gym.load(env_name)
env.reset()

TimeStep(
{'discount': array(1., dtype=float32),
 'observation': array([ 0.0449783 ,  0.0067319 ,  0.02191332, -0.00923979], dtype=float32),
 'reward': array(0., dtype=float32),
 'step_type': array(0)})

In the Cartpole environment:

observation is an array of 4 floats: <br>
the position and velocity of the cart <br>
the angular position and velocity of the pole <br>
reward is a scalar float value <br>
action is a scalar integer with only two possible values: <br>
0 — "move left" <br>
1 — "move right" <br>

In [4]:
print('Observation Spec:')
print(env.time_step_spec().observation)

Observation Spec:
BoundedArraySpec(shape=(4,), dtype=dtype('float32'), name='observation', minimum=[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], maximum=[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38])


In [5]:
print('Reward Spec:')
print(env.time_step_spec().reward)

Reward Spec:
ArraySpec(shape=(), dtype=dtype('float32'), name='reward')


In [6]:
print('Action Spec:')
print(env.action_spec())

Action Spec:
BoundedArraySpec(shape=(), dtype=dtype('int64'), name='action', minimum=0, maximum=1)


In [30]:
env.time_step_spec()

TimeStep(
{'discount': BoundedArraySpec(shape=(), dtype=dtype('float32'), name='discount', minimum=0.0, maximum=1.0),
 'observation': BoundedArraySpec(shape=(4,), dtype=dtype('float32'), name='observation', minimum=[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], maximum=[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]),
 'reward': ArraySpec(shape=(), dtype=dtype('float32'), name='reward'),
 'step_type': ArraySpec(shape=(), dtype=dtype('int32'), name='step_type')})

In [10]:
time_step = env.reset()
print('Time step:')
print(time_step)

action = np.array(1, dtype=np.int32)

next_time_step = env.step(action)
print('Next time step:')
print(next_time_step)

Time step:
TimeStep(
{'discount': array(1., dtype=float32),
 'observation': array([ 0.01751985,  0.04434055, -0.00962096, -0.03994654], dtype=float32),
 'reward': array(0., dtype=float32),
 'step_type': array(0)})
Next time step:
TimeStep(
{'discount': array(1., dtype=float32),
 'observation': array([ 0.01840666,  0.23959914, -0.01041989, -0.33564937], dtype=float32),
 'reward': array(1., dtype=float32),
 'step_type': array(1)})


In [12]:
fc_layer_params = (100, 50)
action_tensor_spec = tensor_spec.from_spec(env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

In [13]:
action_tensor_spec

BoundedTensorSpec(shape=(), dtype=tf.int64, name='action', minimum=array(0, dtype=int64), maximum=array(1, dtype=int64))

In [14]:
num_actions

2

In [24]:
py_env = suite_gym.load(env_name)
env = tf_py_environment.TFPyEnvironment(py_env)

In [25]:
# Define a helper function to create Dense layers configured with the right
# activation and kernel initializer.
def dense_layer(num_units):
    return tf.keras.layers.Dense(
        num_units,
        activation=tf.keras.activations.relu,
        kernel_initializer=tf.keras.initializers.VarianceScaling(
            scale=2.0, mode='fan_in', distribution='truncated_normal')
    )

# QNetwork consists of a sequence of Dense layers followed by a dense layer
# with `num_actions` units to generate one q_value per available action as
# its output.
dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential(dense_layers + [q_values_layer])

In [26]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    env.time_step_spec(),
    env.action_spec(),
    q_network=q_net,
    optimizer=optimizer)

agent.initialize()

In [27]:
q_net.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              multiple                  500       
_________________________________________________________________
dense_7 (Dense)              multiple                  5050      
_________________________________________________________________
dense_8 (Dense)              multiple                  102       
Total params: 5,652
Trainable params: 5,652
Non-trainable params: 0
_________________________________________________________________
